In [2]:
# Reference: http://mohitmayank.com/a_lazy_data_science_guide/natural_language_processing/GPTs/#finetuning-gpt-2-for-sentiment-classification

import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name= "gpt2"

In [3]:
# loading the tokenizer and model
tokenizer= GPT2Tokenizer.from_pretrained(model_name)
model= GPT2LMHeadModel.from_pretrained(model_name).cuda()

c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AIXI\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [49]:
# create prompt
prompt = """

Question: What is wireless communication?
Category: Communication

Question: Is BJT is a three terminal device?
Category: Electronics

Question: What are different modes of propagation?
"""

In [50]:
generated= tokenizer(prompt, return_tensors="pt").input_ids.cuda()

In [51]:
sample_output= model.generate(generated, do_sample=False, top_k=50, max_length=112, top_p=0.90, temperature=0.5, num_return_sequences=0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [52]:
#decoding the predicted tokens into text
predicted_text= tokenizer.decode(sample_output[0], skip_special_tokens=True)

In [53]:
print(predicted_text)



Question: What is wireless communication?
Category: Communication

Question: Is BJT is a three terminal device?
Category: Electronics

Question: What are different modes of propagation?

Category: Electronics

Question: What is the difference between a wireless and a two terminal device?

Category: Electronics

Question: What is the difference between a wireless and a two terminal device?

Category: Electronics

Question: What is the difference between a wireless and a two terminal device?

Category: Electronics


### Finetuning on sentiment analysis task

In [54]:
import re
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

In [119]:
class SentimentDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        self.input_ids=[]
        self.attn_masks=[]
        self.labels=[]
        map_label= {0:'negative', 4:'positive'}
        #iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            #prepare the text
            prep_txt= f'<|startoftext|>Tweet:{txt}<|pad|>Sentiment:{map_label[label]}<|endoftext|>'
            #tokenize
            encodings_dict= tokenizer(prep_txt, truncation=True, max_length=max_length, padding="max_length")

            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]



In [120]:
# data load function
def load_sentiment_dataset(tokenizer):
    # load dataset and sample 10k reviews
    file_path= "noemoticon.csv"
    df= pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    df= df[[0, 5]]
    df.columns= ['label', 'text']
    df= df.sample(10000, random_state=1)

    # divide our dataset into train and test set
    X_train, X_test, y_train, y_test= train_test_split(df['text'].tolist(), df['label'].tolist(), shuffle=True, test_size=0.05,
                                                         random_state=1, stratify= df['label'])
    
    # format into SentimentDataset class
    train_dataset= SentimentDataset(X_train, y_train, tokenizer, max_length=512)
    return train_dataset, (X_test, y_test)


In [121]:
# lOad model and data

model_name= 'gpt2'
torch.manual_seed(42)

# load tokenizer and model
tokenizer= GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token= '<|pad|>')
model= GPT2LMHeadModel.from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))

loading file vocab.json from cache at C:\Users\AIXI/.cache\huggingface\hub\models--gpt2\snapshots\11c5a3d5811f50298f278a704980280950aedb10\vocab.json
loading file merges.txt from cache at C:\Users\AIXI/.cache\huggingface\hub\models--gpt2\snapshots\11c5a3d5811f50298f278a704980280950aedb10\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\AIXI/.cache\huggingface\hub\models--gpt2\snapshots\11c5a3d5811f50298f278a704980280950aedb10\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner

Embedding(50259, 768)

In [122]:
# prepare and load dataset
train_dataset, test_dataset= load_sentiment_dataset(tokenizer)

# creating training arguments
training_args= TrainingArguments(
                output_dir='results',
                num_train_epochs=2, 
                logging_steps=10,
                load_best_model_at_end=True,
                save_strategy="epoch",
                evaluation_strategy="epoch",
                per_device_train_batch_size=2,
                per_device_eval_batch_size=2,
                warmup_steps=100,
                weight_decay=0.01,
                logging_dir='logs')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [124]:
# Starting training

Trainer(model= model, 
        args= training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator= lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                    'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack([f[0] for f in data])}).train()

***** Running training *****
  Num examples = 9500
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9500
  Number of trainable parameters = 124441344


  0%|          | 0/9500 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\AIXI\AppData\Local\Temp\ipykernel_11592\837832459.py:9 in <module>                      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\AIXI\\AppData\\Local\\Temp\\ipykernel_11592\\837832459.py'                           │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:1527 in train      │
│                                                                                                  │
│   1524 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1525 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1526 │   │   )                                                                                 │
│ ❱ 1527 │   │   return inner_training_loop(                                                       │
│   1528 │   │   │   args=args,                                                                    │
│   1529 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1530 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:1775 in            │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1772 │   │   │   │   │   with model.no_sync():                                                 │
│   1773 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1774 │   │   │   │   else:                                                                     │
│ ❱ 1775 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1776 │   │   │   │                                                                             │
│   1777 │   │   │   │   if (                                                                      │
│   1778 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:2523 in            │
│ training_step                                                                                    │
│                                                                                                  │
│   2520 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2521 │   │                                                                                     │
│   2522 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2523 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2524 │   │                                                                                     │
│   2525 │   │   if self.args.n_gpu > 1:                                                           │
│   2526 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ c:\Users\AIXI\anaconda3\envs\newenv\lib\site-packages\transformers\trainer.py:2555 in            │
│ compute_loss                                               